### Use the code in this notebook to train the machine learning model on Azure Machine Learning Workspace

In [ ]:
'''Don't change the code in here'''

# import library 
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline 

# import machine learning library 
from azureml.core import Experiment, Run, Workspace
import azureml.core
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

### <font color='red'>Fill in details in the cell below</font>

In [ ]:
# connect to Azure database
import pyodbc
server = ''
database = ''
username = ''
password = ''
driver= '{ODBC Driver 17 for SQL Server}' #don't change this

### <font color='red'>Fill in details in the cell below as instructed</font>

In [ ]:
# connect to Azure Machine Learning Workspace
subscription_id = ''
resource_group  = '' # name of resource_group for ML workspace
workspace_name  = ''

In [ ]:
'''Don't change the code in here'''

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')

### Export database (description, date, quantity) to csv 
### Store it in the same directory (default location) as this notebook 
### Run the code below to process the raw data
### A file named 'date_description_quantity.csv' should be generated

In [ ]:
'''Don't change the code in here'''

# export to csv 
import sys
import csv

export_query='SELECT order_date, order_description_and_pack_size, order_quantity FROM combined_date_formated_dataframe;'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute(export_query)
result=cursor.fetchall()
column_title = ('date', 'description', 'quantity')

c = csv.writer(open('date_description_quantity.csv', 'w', newline=''))
c.writerow(column_title)
for x in result:
    c.writerow(x)

# close the connection and remove the cursor
cursor.close()
cnxn.close()

In [ ]:
'''Don't change the code in here'''

# define the data preprocess function
def preprocess(df, min_purchase, time_period, end_date):
    from dateutil import parser

    # group by date and description
    df = df.groupby(['description','date'], as_index=False).sum()
    df.sort_values(by = ['description'])

    # count the number of times an item is perchased
    df_2 = df.groupby('description', as_index = False).count()
    df_2.drop(df_2.loc[df_2['quantity'] < min_purchase].index, inplace=True) 
    items = df_2['description'].tolist()
    items.sort()

    # creat new df that contains the filtered out items only
    df_filtered = df[df['description'].isin(items)]
    df_filtered.sort_values(by='date') 

    #create new_df_filtered such that dates are equaly spaced out (this is neccessary for Azure ML to run)

    #disable SettingWithCopyWarning
    pd.options.mode.chained_assignment = None

    #iterate through descriptions and map to new datetime slots
    days = pd.date_range(start='2016-11-10', end=end_date, periods=time_period) 
    concat_df = pd.DataFrame(columns = ['date','quantity','description'])

    for a in items:
        #set up new blank dataframe filled with 0 
        new_df_filtered = pd.DataFrame(columns = ['date','quantity','description'])
        new_df_filtered.loc[:,'date'] = days
        new_df_filtered.fillna(value = 0, inplace = True)
        new_df_filtered['description'] = [a]*time_period  

        #use a dummy_df to store df with the matching description 
        dummy_df = df.loc[df['description'] == a]
        rows = dummy_df.shape[0]
        j = 0 
        for i in dummy_df.index.tolist():
            while parser.parse(dummy_df['date'][i]) > new_df_filtered['date'][j]:
                j = j+1
            new_df_filtered['quantity'][j] = new_df_filtered['quantity'][j] + dummy_df['quantity'][i]

        #append the dataframe of the new item to the old dataframe
        concat_df = pd.concat([concat_df, new_df_filtered], ignore_index=True)
    
    return concat_df, items

In [ ]:
'''Don't change the code in here'''

# define the model training function
def train_model(concat_df, items, ws, run_name = 'testrun2-'):

    #Load automl packages and define the time column name
    from azureml.train.automl import AutoMLConfig
    from azureml.train.automl.constants import Metric
    time_column_name = 'date'


    from azureml.core.experiment import Experiment
    from azureml.widgets import RunDetails

    #count-for naming purposes 
    count = 0

    #iterate through the dataframe by description 
    for a in items:
        count = count + 1 
        df_by_description = concat_df.loc[concat_df['description'] == a]
        df_by_description.sort_values(by=['date','description'], ascending = True, inplace = True)  

        #Save some data back to make sure we are not overfitting
        reserve = 8
        X_train = df_by_description[:-reserve]
        X_test = df_by_description[-reserve:]

        target_column_name = 'quantity'
        y_train = X_train.pop(target_column_name).values

        time_series_settings = {
              "time_column_name": time_column_name,
    #           "grain_column_names": ["description"], 
    #           "target_lags": 2,
    #           "target_rolling_window_size": 5
                }

        #Local compute 
        Automl_config = AutoMLConfig(task = 'forecasting',
                                     primary_metric = 'normalized_root_mean_squared_error',
                                     iteration_timeout_minutes = 3,
                                     iterations = 25, # increase this number for more iteration
                                     n_cross_validations=5,
                                     preprocess = False,
                                     enable_stack_ensemble=False, #ensemble tends to overfit
                                     enable_voting_ensemble=False,
                                     blacklist_models=['DecisionTree'], #decision tree tends to overfit
                                     X = X_train,
                                     y = y_train,
                                     path='',
                                     **time_series_settings)


        #Run the automl using the local notebook free compute
        name = run_name + str(count) + '-' + str(a[:3]) 
        experiment = Experiment(ws, name)
        local_run = experiment.submit(Automl_config, show_output=True)

    return(print('training complete'))

### <font color='red'>Change</font> parameter in the code below to execute 
### <font color='orange'>min_purchase:</font> only create model on items purchased more than 9 times, change this number if wish to model items that are less frequently purchased
### <font color='orange'>time_period:</font> change this number to increase/decrease the number of time slots the date will be divided into
### <font color='orange'>end_date:</font> date of the latest order recorded

##### <font color='red'>time_period = 49 may raise error in the future as end_date changes</font> 
##### currently no better method other than trail_and_error fo identifying a new time_period what will work
##### this problem may be fixed in future version of Azure ML library

#### <font color='red'>if you have changed any of those parameters for training your new model, the data shape would have changed, you should then only use the lateset run in your prediction (i.e. use your latest run in notebook 3)</font> 

In [ ]:
# import csv
df = pd.read_csv('date_description_quantity.csv')

# call function to preprocess thed data
concat_df, items = preprocess(df, min_purchase = 10, time_period = 49, end_date='2019-05-10') # change parameters here

# export concat_df to csv (this csv will be used by notebook '3-predict.ipynb')
export_csv = concat_df.to_csv ('export_concat_df.csv', index = None, header=True)

### <font color='red'>Change</font> <font color='orange'>run_name</font>  here 
### The code below trains the models on Azure, so it may take sometime for the training to complete

In [ ]:
# feed data into model and train models
train_model(concat_df, items, ws, run_name = 'testrun4-') # change run_name